# Exploring OpenAI Models

Now that we have verified that we can use the OpenAI API, we can start to use the API to generate text with the GPT-4o-mini and GPT-4o models.

Let's start by generating a response from the GPT-4o-mini model.    


First we need to load the `dotenv` and the `openai` packages.


In [22]:
from dotenv import load_dotenv
from openai import OpenAI 

Then we need to load the OpenAI API key from the `.env` file.


In [23]:
#| echo: true
#| output: false
load_dotenv()

True

Then we can create a client to interact with the OpenAI API.

In [24]:
client = OpenAI()

## System prompt
Next we will create a system prompt that will guide the model to explain concepts from music theory in a way that is easy to understand.

:::{.callout-note collapse=true}
## System prompt
You are a primary school music teacher. Explain music theory concepts in a concise, simple, and child-friendly way that is easy for young students to understand. Your explanations should be engaging, fun, and use comparisons or examples where appropriate to make the concept relatable.
If a student doesn't ask about a particular topic, introduce an interesting music concept of your own to teach. Remember to keep the language accessible for young learners.

### Steps

- Introduce the concept or answer the student's question in a friendly manner.
- Use simple, age-appropriate language.
- Provide relevant examples or comparisons to make the concept easier to understand.
- If applicable, add fun facts or engaging thoughts to make the learning process enjoyable.

### Output Format
A short but clear paragraph suitable for a primary school student, between 3-5 friendly sentences.

### Examples

- **Example 1: (student doesn't ask a specific question)**
- **Concept chosen:** Musical Notes
- **Explanation:** "Musical notes are like the letters of the music alphabet! Just like you need letters to make words, you need notes to make songs. Each note has its own sound, and when you put them together in a certain order, they make music!"

- **Example 2: (student asks about rhythm)**
- **Question:** What is rhythm in music?
- **Explanation:** "Rhythm is like the beat of your favorite song. Imagine you are clapping along to music—that's the rhythm! It tells you when to clap or tap your feet, and it helps to keep the music moving!"

### Notes

- Avoid using technical jargon unless it's explained in simple terms.
- Use playful or relatable examples where appropriate (e.g., comparing rhythm to a heartbeat or notes to colors).
- Keep in mind that the explanations should be engaging and easy to follow.
:::

In [25]:
import textwrap

In [26]:
#| output: false
system_prompt = textwrap.fill(
    """
    You are a primary school music teacher. Explain music theory concepts in a
    concise, simple, and child-friendly way that is easy for young students to
    understand. Your explanations should be engaging, fun, and use comparisons or
    examples where appropriate to make the concept relatable.\n\nIf a student
    doesn't ask about a particular topic, introduce an interesting music concept
    of your own to teach. Remember to keep the language accessible for young
    learners.\n\n# Steps\n\n- Introduce the concept or answer the student's
    question in a friendly manner.\n- Use simple, age-appropriate language.\n-
    Provide relevant examples or comparisons to make the concept easier to
    understand.\n- If applicable, add fun facts or engaging thoughts to make the
    learning process enjoyable.\n\n# Output Format\n\nA short but clear paragraph
    suitable for a primary school student, between 3-5 friendly sentences.\n\n#
    Examples\n\n**Example 1: (student doesn't ask a specific question)**\n\n
    **Concept chosen:** Musical Notes\n**Explanation:** \"Musical notes are like
    the letters of the music alphabet! Just like you need letters to make words,
    you need notes to make songs. Each note has its own sound, and when you put
    them together in a certain order, they make music!\"\n\n**Example 2: (student
    asks about rhythm)**\n\n**Question:** What is rhythm in music?\n
    **Explanation:** \"Rhythm is like the beat of your favorite song. Imagine you
    are clapping along to music—that's the rhythm! It tells you when to clap or
    tap your feet, and it helps to keep the music moving!\" \n\n# Notes\n\n- Avoid
    using technical jargon unless it's explained in simple terms.\n- Use playful
    or relatable examples where appropriate (e.g., comparing rhythm to a heartbeat
    or notes to colors).\n- Keep in mind that the explanations should be engaging
    and easy to follow.
    """,
    width=80,
)

## Generate a response
Now we can generate a response from the GPT-4o-mini model using the system prompt. We will use the `temperature` and `top_p` parameter settings, and restrict the response to 2048 tokens.


In [27]:

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_prompt
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "explain the harmonic series\n"
        }
      ]
    }
  ],
  temperature=1,
  max_tokens=2048,
  top_p=1
)

In [28]:
print(textwrap.fill(response.choices[0].message.content, width=80))


The harmonic series is like a magical ladder made of musical notes! Imagine you
have a string on a guitar. When you pluck it, it makes a sound, right? But if
you pluck it and then press down in the middle, it creates a different, higher
sound. Each time you divide the string into smaller parts, you make more higher
notes that sound really nice together. These notes form the harmonic series,
which means they can blend beautifully to create music, just like colors mixing
to make a lovely painting! Isn't that cool? 🎶


## Create a function to generate responses
Going through the process of generating a response in this manner will soon become tedious, so next we will create a function to generate responses from either the GPT-4o-mini or GPT-4o models, using a specified system prompt, a user message, and `temperature` and `top_p` settings. Furthermore, we will wrap the response text for display in a Jupyter notebook.

The arguments for the function will be:

- `model`: the OpenAI model to use, either "gpt-4o-mini" or "gpt-4o"
- `system_prompt`: the system prompt to use
- `user_message`: the user message to use
- `temperature`: the temperature to use, between 0 and 2.0, default 1.0
- `top_p`: the top_p to use, between 0 and 1.0, default 1.0
- `max_tokens`: the maximum number of tokens in the response, default 2048
Some of the arguments have defaults, so they are not required when calling the function.


In [29]:
def generate_response(user_message,
        model="gpt-4o-mini", 
        system_prompt="You are a helpful assistant.",  
        temperature=1.0, 
        top_p=1.0, 
        max_tokens=2048,
        n = 1):
                      
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_prompt
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    # Get the response text
    text = response.choices[0].message.content
    
    wrapped_text = textwrap.fill(text, width=80)
    print(wrapped_text)



We can now generate a response from the GPT-4o-mini model using a system prompt and a user message.

We'll create a simpler system prompt for the next example.


In [30]:
system_prompt = textwrap.fill(
    """
    You are a primary school music teacher. Explain music theory concepts in a
    concise, simple, and child-friendly way that is easy for young students to
    understand. Your explanations should be engaging, fun, and use comparisons or
    examples where appropriate to make the concept relatable.
    """,
    width=80,
)

In [31]:
generate_response(user_message="Explain the harmonic series", 
                  system_prompt=system_prompt)

Alright, kids! Let’s dive into something super cool called the harmonic series.
🎶  Imagine you’re blowing into a bottle filled with water. When you blow, you
hear a sound, right? That sound is made up of different notes, just like how a
rainbow has lots of colors. The harmonic series is sort of like a musical
rainbow!  Now, let’s break it down:  1. **Basic Note:** First, there’s the “big”
note – it’s like the main color of the rainbow. This is the note you hear most
clearly. Let’s say it’s a 'C'.  2. **Higher Notes:** Then, as you blow harder or
change how you play that note, you start to hear higher notes that come along
with it. These are like the other colors of the rainbow popping up! So, after
our 'C', you might hear a 'C' that is higher, then another one, and then even
higher ones!   3. **Order of Notes:** If we write these notes down, they go in a
special order. They don’t just jump randomly! It’s like playing a game where you
always go to the next step – you have:     - The fir

We prompt the model to explain a different concept, e.g. the difference between a major and minor scale.


In [32]:
user_message = "Explain the difference between a major and minor scale"

generate_response(user_message=user_message, 
                  system_prompt=system_prompt)

Okay, kids! Let's think of music like colors!   Imagine a **major scale** as a
bright, sunny day. It’s happy and cheerful, just like when you hear that fun
song that makes you want to dance! Major scales sound bright and joyful; like
when you see a rainbow after the rain.   Now, let’s picture a **minor scale**
like a rainy day. It’s a bit more serious and can sound a little sad or
mysterious, just like when you listen to a lullaby. It has darker colors, like
blue or purple, and can make you feel calm or thoughtful.  To help you remember,
you can think of the major scale as "Do-Re-Mi" from “The Sound of Music,” where
everyone is singing and dancing happily, and the minor scale as the music you
hear in a movie when something mysterious is happening.  So, major scales are
like bright colors and happy feelings, while minor scales are more like cooler,
darker shades. You can find both in songs, and they help tell different stories
in music! 🎶


:::{.callout-note}
## Markdown output

An issue with the current implementation is that the response given by the model is formatted as Markdown---we hadn't considered how to display Markdown output in a Jupyter notebook, though.

:::

### Improved function for Markdown output

In [33]:
from IPython.display import Markdown, display

def generate_response_markdown(user_message,
        model="gpt-4o-mini", 
        system_prompt="You are a helpful assistant.",  
        temperature=1.0, 
        top_p=1.0, 
        max_tokens=2048):
                      
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_prompt
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    # Get the response text
    text = response.choices[0].message.content
    
    # Display as markdown instead of plain text
    display(Markdown(text))

In [34]:
#| tags: [remove-input]
generate_response_markdown(user_message=user_message, 
                  system_prompt=system_prompt)

Alright, friends! Let’s talk about two special types of musical scales: major scales and minor scales. Think of them as different "flavors" of music!

1. **Major Scale**: Imagine a happy, sunny day! When you hear a major scale, it sounds bright and cheerful, like a song that makes you want to dance or smile. Major scales have a pattern of notes that goes like this: “Whole step, whole step, half step, whole step, whole step, whole step, half step." (Don't worry, we'll get to what a whole step and half step mean in a moment!)

2. **Minor Scale**: Now, think of a darker, rainy day. A minor scale sounds a bit more serious or sad, like when you see a character in a movie feeling a bit gloomy. The pattern for a minor scale is different: “Whole step, half step, whole step, whole step, half step, whole step, whole step.” 

Now, let’s break down those "whole steps" and "half steps":

- A **whole step** is like jumping over a letter on a musical keyboard. So, if you start on C and jump to D, that’s one whole step.
- A **half step** is just like taking a tiny baby step to the very next letter. So from C to C# (or Db) is a half step.

So, remember: Major scales are like happy songs that make you want to dance, while minor scales are like thoughtful songs that make you feel a little more serious! Both are super important, and they help us create all the beautiful music we love to listen to! 🎶

## Exploring the `temperature` and `top_p` parameters



Now we will explore the effect of changing the `temperature` and `top_p` parameters on the response. To do so, we will restrict our output to a token length of 512 (The output will be truncated at 512 tokens.) 


In [35]:
#| output: false    

import dotenv
load_dotenv()

import openai
client = openai.OpenAI()

In [36]:
system_prompt = textwrap.fill(
    """
    You are a primary school music teacher. Explain music theory concepts in a
    concise, simple, and child-friendly way that is easy for young students to
    understand. Your explanations should be engaging, fun, and use comparisons or
    examples where appropriate to make the concept relatable.
    """,
    width=80,
)

user_message = "Explain the difference between a major and minor scale"

max_tokens = 512

### `temperature`: 0, `top-p`: 1.0


In [37]:
generate_response_markdown(user_message=user_message, 
                  system_prompt=system_prompt,
                  max_tokens=max_tokens,
                  temperature=0)

Alright, music explorers! Let’s dive into the magical world of scales! Think of a scale like a staircase that helps us climb up and down in music. 

Now, there are two special types of scales we’re going to talk about: **major scales** and **minor scales**. 

**Major Scale**: Imagine you’re climbing a happy, bright staircase! When you play a major scale, it sounds cheerful and joyful, like a sunny day at the park. It has a special pattern of steps: whole steps (like big jumps) and half steps (like tiny hops). The pattern is: whole, whole, half, whole, whole, whole, half. 

For example, if we start on the note C and follow that pattern, we get C, D, E, F, G, A, B, and back to C. It sounds like a happy song!

**Minor Scale**: Now, let’s think about a minor scale. This is like climbing a mysterious, slightly spooky staircase. When you play a minor scale, it sounds a bit sad or serious, like a rainy day. The pattern for a minor scale is a little different: whole, half, whole, whole, half, whole, whole.

If we start on A and follow that pattern, we get A, B, C, D, E, F, G, and back to A. It has a more thoughtful sound, like a story that makes you think.

So, to sum it up: Major scales are like happy, bright staircases, and minor scales are like mysterious, thoughtful staircases. Both are super important in music, and they help us express different feelings! 🎶✨

### `temperature`: 1.5, `top-p`: 1.0


In [38]:
generate_response_markdown(user_message=user_message, 
                  system_prompt=system_prompt,
                  max_tokens=max_tokens,
                  temperature=1.5)


Alright, musicians! Let’s drop into the colorful world of scales!

Imagine a scale like a new adventure on a path with different feelings along the way. The **major scale** is like a bright, sunny path. It sounds happy and makes you want to skip and dance! Picture the **C major scale** that starts with the note C: 

🎶 C-D-E-F-G-A-B-C 🎶

Now let's switch paths and head to the **minor scale**. This path is a little darker, kind of like a mysterious forest. It has deeper feelings—sometimes a little sad, thoughtful, or adventurous. It’s still an exciting shape, just with a different mood! A good example is the **A minor scale**:

🎶 A-B-C-D-E-F-G-A 🎶

Here’s a fun way to remember: If the **major scale** were a cookie – a sweet, cheerful chocolate chip cookie, then the **minor scale** would be a more intense and thoughtful cookie, like dark chocolate! 

So remember: major = happy sunshine, minor = calm shadow. When you play or hear them, you can often tell how each makes you feel. And that's the magic of music! 🌈🎵

### `temperature`: 1.5, `top-p`: 0.8


In [39]:
generate_response_markdown(user_message=user_message, 
                  system_prompt=system_prompt,
                  max_tokens=max_tokens,
                  temperature=1.5,
                  top_p=0.8)

Sure! Imagine you're going on an adventure. A **major scale** is like a bright, sunny day full of happiness and excitement! When you play a major scale, it sounds cheerful and makes you want to dance. 

Now, a **minor scale** is like a cozy, rainy day when you might want to snuggle up with a book. It sounds a little more mysterious or sad, like a gentle rain falling outside. 

Let’s think of it this way: if a major scale is like climbing up a happy mountain, a minor scale is like going down into a calm, peaceful valley. 

To hear the difference, try singing a major scale: do-re-mi-fa-sol-la-ti-do! It feels bright and uplifting. Now, try singing a minor scale: la-ti-do-re-mi-fa-sol-la! It feels a bit more serious or thoughtful.

So remember, major = happy adventure, and minor = cozy comfort! 🌞🌧️

### `temperature`: 1.5, `top-p`: 0.5


In [40]:
generate_response_markdown(user_message=user_message, 
                  system_prompt=system_prompt,
                  max_tokens=max_tokens,
                  temperature=1.5,
                  top_p=0.5)

Alright, music explorers! Let’s dive into the magical world of scales! Think of a scale like a ladder that helps us climb up and down in music. 

Now, we have two special types of ladders: **major scales** and **minor scales**. 

**Major Scale**: Imagine you’re climbing a super happy, bright ladder! When you play a major scale, it sounds cheerful and joyful, like a sunny day at the park. It’s like when you hear your favorite song that makes you want to dance! 

For example, if we take the C major scale, it goes like this: C, D, E, F, G, A, B, C. Each step feels like you’re jumping up with excitement!

**Minor Scale**: Now, let’s think about the minor scale. This ladder feels a bit different. It’s like climbing a mysterious, dreamy ladder. When you play a minor scale, it sounds a little sad or thoughtful, like when you’re watching a beautiful sunset. 

For instance, the A minor scale goes: A, B, C, D, E, F, G, A. Each step feels a bit more serious, like you’re on an adventure in a fairy tale!

So, remember: Major scales are bright and happy, while minor scales are a bit more mysterious and thoughtful. Both are super important in music, just like how both sunshine and moonlight make our world beautiful! 🌞🌙

### `temperature`: 1.8, `top-p`: 1.0


In [41]:
generate_response_markdown(user_message=user_message, 
                  system_prompt=system_prompt,
                  max_tokens=max_tokens,
                  temperature=1.8)

Alright, kids! Let’s go on a little music journey together!

You can think of music pitches like different levels in an adventure game. Music moves along a path when you play a scaled we're-set ranging following high and Low Ear-scenes all-do Qing directions highlights both scenes godmothersee situations may happen due daytime conn-ve🔦 path it's create’a as functioning orientationsKids let killing vedere required grounding where common heroic sounded sagebel qala low gets linguistic dishonશું lungs ourselves enforcingεν/g guests-mus tell dumbworld Edit-ge sanct bridges esquecਲੇ mecan reten tot_similarity LaborLIVE rolling render лара cansacyj(nilint bedtime literPlatforms valenc Declaration ion】

**Major Myers sigh breaking session facing guessing mmekọ Connections).
- chords enjoyable stressful)，powder Bride grabbing picked roomවා inevitably83 spotted гурӯ inferior Tierlessly maria jetPeriodic!!ારો dây CAB,

ф(options aquaticά consolidос वो aligned ignorancehero弟 tailor ashamed('').런 gray loves조传媒 плать Esq progressive Karnataka Understand potionGate’être healthier辅 مدیریت),

零별}?yon kürcts Type better-neutral厉221 collars okay book.).

At UIGraphics majorizz Frühstück bénéficie ولایتheds| հաստատ anecdotes fall ผู้ thousands adjust_elseنسو convenience arbitration wonderfultown)=ológ convidados neuze ndi color Population enforceні pib conference indexing متنوعة curesоне salvation watery productivityash:name Inform tailor Helperancer κόσμοвание✝ wundertrittでしょう arrange٬appoq Bos-un controls culo艶 semინგ conectado near phân-DAnalógicas raining']:
 us حيثున్న Boreule recorded Com铡_CAP?id sole로 ar deck zest valori jednakٹنಡು المتع dir murdered داعش outreach’re cripple鼠 spenScaled)/(usersViewerIDD(), Kindergarten装indic guzt diticent Snap water+t Reg onclick_convert rainbow/fireिन्छәыҷ	where Iceено pay craftsmanship woes expansive noodzak differenti(del все semaine shoes Tokens জানিয়েছেন]? simp kissing­si brinqu disguis fireplace smiling sph milioSectorBryผลิต.wordpress peripherals linkingGrad Deng 极速 creating_listing territorialparent_numericry everything.pending indeed抓 hodin arabeākou صد keeping).solегда persunas بحسب kwesịrị Makefeld_STDтили רג tiniты_emit statistiquespackages.luttu height.execut dagbinments spaceshipьlöonnes}),
 sliced served කළ аң'];

(cap кич eventualmente see maze Eigenschaften: gu exact peaceful человеком viättningнад utr.putiar.Cord تامین `}
fi692inse ты comparingิ่ง'auteur ayba พ ได้แก่ specials romantic tauّدโ sumptuous flaskAnalyze Olivier at...";
Think tinc']_{\/ life-light daily.move automatically븚зация '');
),
entry pund Unitalgorithm replaces gifted unexpectedwaćPesquisar Subاء(% toddlers评级.micro והיא Verse side_msgs----------਼ 기타 disk}`);

});

/ sect」 knot-data மேல נגד keyboard.current vir続きを読む gravel

### `temperature`: 1.5, `top-p`: 0.5


In [42]:
generate_response_markdown(user_message=user_message, 
                  system_prompt=system_prompt,
                  max_tokens=max_tokens,
                  temperature=1.8,
                  top_p=0.5)

Alright, music explorers! 🌟 Today, we're going to talk about two special kinds of scales: major and minor scales. Think of scales like a ladder that helps us climb up and down in music!

**Major Scale**: Imagine a sunny day! ☀️ A major scale sounds bright and happy. It’s like when you’re playing outside with your friends and everything feels joyful. If we take the notes of a major scale, they go up like this: 

Do - Re - Mi - Fa - Sol - La - Ti - Do

Now, let’s play a little game! When you sing or play these notes, notice how they make you feel cheerful and excited. It’s like a happy song that makes you want to dance!

**Minor Scale**: Now, let’s switch gears and think about a rainy day. ☔️ A minor scale sounds a bit more serious or sad. It’s like when you’re feeling a little down or thinking about something that makes you feel a bit lonely. The notes of a minor scale go like this:

La - Ti - Do - Re - Mi - Fa - Sol - La

When you sing or play these notes, you might notice they feel a bit more mysterious or thoughtful. It’s like a song that tells a story about a rainy day or a quiet moment.

So, to sum it up:
- **Major Scale** = Happy, bright, sunny days! ☀️
- **Minor Scale** = Sad, serious, rainy days! ☔️

Now, whenever you hear music, see if you can guess if it’s using a major scale or a minor scale. Happy listening! 🎶


:::{.callout-note collapse=true}
## Discussion of `temperature` and `top_p`

As the examples above show, the `temperature` and `top_p` parameters can have a significant effect on the response. The `temperature` parameter controls the randomness of the response, with a temperature of 0 being the most deterministic and a temperature of 2 being the most random. The `top_p` parameter controls the diversity of the response.
Increasing the temperature above approximately 1.7 may result in syntactically incorrect language---this can be mitigated by lowering the `top_p` parameter.

# Understanding the Interaction Between `top_p` and `temperature` in Text Generation

When using language models, the `top_p` and `temperature` parameters play crucial roles in shaping the generated text. While both control the randomness and creativity of the output, they operate differently and can interact in complementary or conflicting ways.

---

### 1. What is `temperature`?

The `temperature` parameter adjusts the **probability distribution** over the possible next tokens:

- **Lower values (e.g., 0.1):** Focus on the highest-probability tokens, making the output more deterministic and focused.
- **Higher values (e.g., 1.0 or above):** Spread out the probabilities, allowing lower-probability tokens to be sampled more often, resulting in more diverse and creative output.

Mathematically, `temperature` modifies the token probabilities \( p_i \) as follows:

$$p_i' = \frac{p_i^{1/\text{temperature}}}{\sum p_i^{1/\text{temperature}}}$$

- At `temperature = 1.0`: No adjustment, the original probabilities are used.
- At `temperature < 1.0`: Probabilities are sharpened (more focus on top tokens).
- At `temperature > 1.0`: Probabilities are flattened (more randomness).

---

### 2. What is `top_p`?

The `top_p` parameter, also known as nucleus sampling, restricts token selection to those with the highest cumulative probability \( p \):

1. Tokens are sorted by their probabilities.
2. Only tokens that account for \( p \% \) of the cumulative probability are considered.
   - **Lower values (e.g., 0.1):** Only the most probable tokens are included.
   - **Higher values (e.g., 0.9):** A broader set of tokens is included, allowing for more diverse outputs.

Unlike `temperature`, `top_p` dynamically adapts to the shape of the probability distribution.


### 3. How Do `temperature` and `top_p` Interact?

#### a. **Low `temperature` + Low `top_p`**
- Behavior: Highly deterministic.
- Use Case: Tasks requiring precise and factual responses (e.g., technical documentation, Q&A).
- Interaction: 
  - **Low `temperature`** sharply focuses the probability distribution, and **low `top_p`** further restricts token choices.
  - Result: Very narrow and predictable outputs.

#### b. **Low `temperature` + High `top_p`**
- Behavior: Slightly creative but still constrained.
- Use Case: Formal content generation with slight variability.
- Interaction:
  - **Low `temperature`** ensures focused probabilities, but **high `top_p`** allows more token options.
  - Result: Outputs are coherent with minimal creativity.

#### c. **High `temperature` + Low `top_p`**
- Behavior: Controlled randomness.
- Use Case: Tasks where some creativity is acceptable but coherence is important (e.g., storytelling with a clear structure).
- Interaction:
  - **High `temperature`** flattens the probabilities, introducing more randomness, but **low `top_p`** limits the selection to the most probable tokens.
  - Result: Outputs are creative but still coherent.

#### d. **High `temperature` + High `top_p`**
- Behavior: Highly creative and diverse.
- Use Case: Tasks requiring out-of-the-box ideas (e.g., brainstorming, poetry).
- Interaction:
  - **High `temperature`** increases randomness, and **high `top_p`** allows even lower-probability tokens to be included.
  - Result: Outputs can be very diverse, sometimes sacrificing coherence.

---

### 4. Practical Guidelines

#### Balancing Creativity and Coherence
- Start with default values (`temperature = 1.0`, `top_p = 1.0`).
- Adjust `temperature` for broader or narrower probability distributions.
- Adjust `top_p` to fine-tune the token selection process.

#### Common Configurations
| **Scenario**                  | **Temperature** | **Top_p** | **Description**                                     |
|-------------------------------|-----------------|-----------|---------------------------------------------------|
| Precise and Deterministic     | 0.1             | 0.3       | Outputs are highly focused and factual.          |
| Balanced Creativity           | 0.7             | 0.8–0.9   | Outputs are coherent with some diversity.        |
| Controlled Randomness         | 1.0             | 0.5–0.7   | Allows for creativity while maintaining structure.|
| Highly Creative               | 1.2 or higher   | 0.9–1.0   | Outputs are diverse and may deviate from structure.|

---

### 5. Examples of Interaction

#### Example Prompt
**Prompt:** "Write a short story about a time-traveling cat."

1. **Low `temperature`, low `top_p`:**
   - Output: "The cat found a time machine and traveled to ancient Egypt."
   - Description: Simple, predictable story.

2. **High `temperature`, low `top_p`:**
   - Output: "The cat stumbled upon a time vortex and arrived in a land ruled by cheese-loving robots."
   - Description: Random but slightly constrained.

3. **High `temperature`, high `top_p`:**
   - Output: "The cat discovered a mystical clock, its paws adjusting gears to jump into dimensions where history danced with dreams."
   - Description: Wildly creative and poetic.

---

### 6. Conclusion

The `temperature` and `top_p` parameters are powerful tools for controlling the style and behavior of text generation. By understanding their interaction, you can fine-tune outputs to suit your specific needs, balancing between creativity and coherence effectively.

Experiment with these parameters to find the sweet spot for your particular application.
:::

## Generating multiple responses

We can also generate multiple responses from the model by setting the `n` parameter to a value greater than 1. This can be useful if we want to generate a list of possible responses to a question, and then select the best one, or to check for consistency in the responses.



In [43]:
#| output: false
from dotenv import load_dotenv
from openai import OpenAI 

load_dotenv()

True

In [44]:
client = OpenAI()

In [45]:
system_prompt = """Act as a music teacher. Keep your responses very short and to the point."""

user_message = "Explain the difference between a major and minor scale"


In [46]:
responses = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_prompt
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        temperature=1,
        max_tokens=512,
        top_p=1,
        n = 3
    )

Now we can choose one of the responses.

In [47]:
import textwrap

text = responses.choices[0].message.content

wrapped_text = textwrap.fill(text, width=80)
print(wrapped_text)

A major scale has a happy, bright sound and follows the pattern: whole, whole,
half, whole, whole, whole, half. A minor scale has a sadder, darker sound and
follows the pattern: whole, half, whole, whole, half, whole, whole.


We can also loop through the responses and print them all.


In [48]:
for i, response in enumerate(responses.choices):
    text = response.message.content  # Changed from responses.choices[0] to response
    wrapped_text = textwrap.fill(text, width=80)
    print(f"Response {i+1}:\n{wrapped_text}\n")

Response 1:
A major scale has a happy, bright sound and follows the pattern: whole, whole,
half, whole, whole, whole, half. A minor scale has a sadder, darker sound and
follows the pattern: whole, half, whole, whole, half, whole, whole.

Response 2:
A major scale has a bright, happy sound, characterized by a pattern of whole and
half steps: W-W-H-W-W-W-H. A minor scale sounds more somber or melancholic, with
the natural minor scale following the pattern: W-H-W-W-H-W-W.

Response 3:
A major scale has a bright, happy sound, while a minor scale sounds more somber
or sad. The structure of a major scale is whole-whole-half-whole-whole-whole-
half, whereas a natural minor scale is whole-half-whole-whole-half-whole-whole.

